In [1]:
 
import os
import json
import glob
import shutil
from tqdm import tqdm
import cv2


# 1. PATH CONFIGURATION
RAW_ROOT   = "/kaggle/input/car-damage-labeled-yolo"   # Original Kaggle dataset path
SPLIT_ROOT = "/kaggle/working/split_data"              # Output path for YOLO

# Create directories for Train and Val (images & labels)
for sub in ["train/images", "train/labels", "val/images", "val/labels"]:
    os.makedirs(os.path.join(SPLIT_ROOT, sub), exist_ok=True)

# Check content of input folder
print("Content of RAW_ROOT:")
for p in os.listdir(RAW_ROOT):
    print(" -", p)


# 2. CLASS MAPPING
# Map JSON class names to YOLO IDs
CLASS_MAP = {
    "rach": 0,
    "tray_son": 1,
    "vo_kinh": 2,
    "mat_bo_phan": 3,
    "thung": 4,
    "mop_lom": 5,
    "be_den": 6,
}


# 3. HELPER FUNCTIONS

def find_image_path(filename):
    """Find the image path inside RAW_ROOT based on the filename."""
    matches = glob.glob(os.path.join(RAW_ROOT, "**", filename), recursive=True)
    if not matches:
        return None
    return matches[0]

def process_json(json_path, split):
    """
    Read specific JSON (Train/Val), copy images to SPLIT_ROOT, and create YOLO labels.
    """
    print(f"\n Processing JSON: {json_path} -> split='{split}'")

    with open(json_path, "r") as f:
        data = json.load(f)

    # Convert to list if data is a dictionary 
    if isinstance(data, dict):
        entries = list(data.values())
    else:
        entries = data

    # Define output directories
    img_out_dir = os.path.join(SPLIT_ROOT, split, "images")
    lbl_out_dir = os.path.join(SPLIT_ROOT, split, "labels")

    image_count = 0
    box_count = 0

    for entry in tqdm(entries, desc=f"{split} JSON entries"):
        filename = entry.get("name", "")
        if not filename:
            continue

        # Find source image
        img_src = find_image_path(filename)
        if not img_src:
            # Skip if image is not found
            continue

        # Read image to get dimensions
        img = cv2.imread(img_src)
        if img is None:
            continue
        h_img, w_img, _ = img.shape

        # Copy image to destination
        img_dest = os.path.join(img_out_dir, os.path.basename(img_src))
        shutil.copy2(img_src, img_dest)

        # Prepare label file path (.txt)
        txt_dest = os.path.splitext(os.path.join(lbl_out_dir, os.path.basename(img_src)))[0] + ".txt"
        yolo_lines = []

        regions = entry.get("regions", [])
        for region in regions:
            all_x = region.get("all_x", [])
            all_y = region.get("all_y", [])
            class_name = region.get("class", "rach")  # default class

            if not all_x or not all_y:
                continue
            if class_name not in CLASS_MAP:
                # Skip unknown classes
                continue
            
            class_id = CLASS_MAP[class_name]

            # Calculate bounding box (min/max)
            x_min, x_max = min(all_x), max(all_x)
            y_min, y_max = min(all_y), max(all_y)

            width  = x_max - x_min
            height = y_max - y_min
            cx = x_min + width  / 2
            cy = y_min + height / 2

            # Normalize coordinates (0-1 range for YOLO)
            x_c = max(0, min(1, cx / w_img))
            y_c = max(0, min(1, cy / h_img))
            w_n = max(0, min(1, width  / w_img))
            h_n = max(0, min(1, height / h_img))

            # Add to list
            yolo_lines.append(f"{class_id} {x_c:.6f} {y_c:.6f} {w_n:.6f} {h_n:.6f}")
            box_count += 1

        # Write label file if we have valid boxes
        if yolo_lines:
            with open(txt_dest, "w") as f_lbl:
                f_lbl.write("\n".join(yolo_lines))
            image_count += 1

    print(f"✅ {split}: Created {image_count} images and {box_count} boxes.")


# 4. EXECUTION

# Define JSON paths
train_json = os.path.join(RAW_ROOT, "0Train_via_annos.json")
val_json   = os.path.join(RAW_ROOT, "0Val_via_annos.json")

# Validate files exist
if not os.path.exists(train_json) or not os.path.exists(val_json):
    raise FileNotFoundError("JSON files not found (0Train_via_annos.json / 0Val_via_annos.json). Check dataset structure.")

# Run processing
process_json(train_json, "train")
process_json(val_json, "val")

# Print final stats
print("\n--- Final Status ---")
print("Train images:", len(os.listdir(os.path.join(SPLIT_ROOT, "train/images"))))
print("Train labels:", len(os.listdir(os.path.join(SPLIT_ROOT, "train/labels"))))
print("Val images:",   len(os.listdir(os.path.join(SPLIT_ROOT, "val/images"))))
print("Val labels:",   len(os.listdir(os.path.join(SPLIT_ROOT, "val/labels"))))

Content of RAW_ROOT:
 - 0Val_via_annos.json
 - validation
 - 0Train_via_annos.json
 - image

 Processing JSON: /kaggle/input/car-damage-labeled-yolo/0Train_via_annos.json -> split='train'


train JSON entries:  14%|█▍        | 1672/11621 [03:50<14:01, 11.83it/s]Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
train JSON entries:  50%|█████     | 5863/11621 [11:24<08:28, 11.31it/s]  Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
train JSON entries:  50%|█████     | 5865/11621 [11:24<08:27, 11.33it/s]Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
train JSON entries:  79%|███████▉  | 9230/11621 [17:28<03:16, 12.16it/s]Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 149 extraneous bytes before marker 0xce
train JSON entries: 100%|██████████| 11621/11621 [21:39<00:00,  8.94it/s]


✅ train: Created 11621 images and 30046 boxes.

 Processing JSON: /kaggle/input/car-damage-labeled-yolo/0Val_via_annos.json -> split='val'


val JSON entries:  77%|███████▋  | 1794/2324 [03:27<01:24,  6.26it/s]Corrupt JPEG data: premature end of data segment
Premature end of JPEG file
val JSON entries: 100%|██████████| 2324/2324 [04:30<00:00,  8.60it/s]

✅ val: Created 2324 images and 6035 boxes.

--- Final Status ---
Train images: 11621
Train labels: 11621
Val images: 2324
Val labels: 2324


In [2]:
import yaml
import os


# CONFIGURATION SETUP
DATA_CFG = "/kaggle/working/damage_config.yaml"

# Define the dictionary structure for YOLO
config = {
    "path": "/kaggle/working/split_data",  # Root folder for the dataset
    "train": "train/images",             # Location of training images
    "val": "val/images",                 # Location of validation images
    "nc": 7,                             # Number of classes (Total: 7)
    
    # English names mapping to IDs (0-6)
    "names": [
        "tear_crack",       # 0 
        "scratch",          # 1 
        "shattered_glass",  # 2  
        "missing_part",     # 3  
        "puncture",         # 4 
        "dent",             # 5 
        "broken_light"      # 6 
    ]
}

# SAVE YAML FILE
with open(DATA_CFG, "w") as f:
    yaml.dump(config, f)

print(f"Config saved successfully: {DATA_CFG}")
print("\n--- YAML File Content ---")
print(open(DATA_CFG).read())

Config saved successfully: /kaggle/working/damage_config.yaml

--- YAML File Content ---
names:
- tear_crack
- scratch
- shattered_glass
- missing_part
- puncture
- dent
- broken_light
nc: 7
path: /kaggle/working/split_data
train: train/images
val: val/images



In [3]:

!pip install ultralytics

from ultralytics import YOLO
import torch


# SYSTEM CHECK (GPU)
print("CUDA Available :", torch.cuda.is_available())
print("GPU Count      :", torch.cuda.device_count())

# List all available GPUs
for i in range(torch.cuda.device_count()):
    print(f" - GPU {i}:", torch.cuda.get_device_name(i))


# INITIALIZE MODEL
# Load a pretrained model
model = YOLO("yolov8s.pt")


# 4. START TRAINING
results = model.train(
    data="/kaggle/working/damage_config.yaml", # Path to config file from Step 2
    epochs=50,                  # Number of training cycles
    imgsz=640,                  # Image resolution
    batch=16,                   # Batch size (images per step)
    name="car_damage_7cls_final", # Folder name for saving results
    patience=10,                # Stop early if no improvement
    device=0,                   # Use GPU 0
    workers=2,                  # Low worker count for Kaggle stability
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mkl-fft to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-random to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-umath to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 113.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       2322       6030      0.594      0.458      0.479      0.299
            tear_crack        659        963       0.45      0.308      0.294      0.169
               scratch       1140       2378       0.47      0.291      0.294      0.156
       shattered_glass        379        396      0.831      0.725      0.804      0.587
          missing_part        402        448      0.685      0.612      0.661      0.472
              puncture        317        416      0.588       0.49       0.48      0.292
                  dent        749        971      0.458      0.237      0.256      0.114
          broken_light        406        458      0.676      0.542      0.561      0.307
Speed: 0.1ms preprocess, 3.3ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to /kaggle/working/runs/detect/car_damage_7cls_final


In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import os

# 1. LOAD BEST MODEL
# Load the best weights saved during training
model_path = "/kaggle/working/runs/detect/car_damage_7cls_final/weights/best.pt"

if not os.path.exists(model_path):
    print(f"WARNING: Model file not found at {model_path}")
else:
    model = YOLO(model_path)
    print("Model loaded successfully.")

# 2. RUN PREDICTIONS
print("Starting prediction on validation data...")

results = model.predict(
    source="/kaggle/working/split_data/val/images", # Test images folder
    save=True,    # Save the images with bounding boxes
    conf=0.25,    # Confidence threshold (Only accept > 25% sure)
    project="/kaggle/working/runs/detect", # Save location root
    name="predictions", # Save folder name (creates runs/detect/predictions)
    exist_ok=True # Overwrite if folder exists
)

print("\n Results saved to: /kaggle/working/runs/detect/predictions")

Model loaded successfully.
Starting prediction on validation data...

WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/2324 /kaggle/working/split_data/val/images/01022020_104459image894113.jpg: 480x640 3 tear_cracks, 3 scratchs, 7.9ms
image 2/2324 /kaggle/working/split_data/val/images/01022020_104510image892040.jpg: 480x640 2 tear_cracks, 1 scratch, 2 punctures, 6.5ms
image 3/2324 /kaggle/working/split_data/val/images/01022020_153342image229797.jpg: 480x640 1 shattered_glass, 1 missing_part, 6.4

Corrupt JPEG data: premature end of data segment


image 1805/2324 /kaggle/working/split_data/val/images/24112020_094415image784831.jpg: 480x640 1 shattered_glass, 7.0ms
image 1806/2324 /kaggle/working/split_data/val/images/24112020_094737image148634.jpg: 480x640 1 shattered_glass, 6.7ms
image 1807/2324 /kaggle/working/split_data/val/images/24112020_095017image896092.jpg: 480x640 1 scratch, 1 shattered_glass, 7.2ms
image 1808/2324 /kaggle/working/split_data/val/images/24112020_103352image905652.jpg: 480x640 1 shattered_glass, 6.7ms
image 1809/2324 /kaggle/working/split_data/val/images/24112020_150848image747745.jpg: 480x640 1 shattered_glass, 6.8ms
image 1810/2324 /kaggle/working/split_data/val/images/24122020_101520image987569.jpg: 480x640 1 scratch, 1 dent, 1 broken_light, 6.7ms
image 1811/2324 /kaggle/working/split_data/val/images/24122020_102851image478211.jpg: 480x640 3 scratchs, 1 puncture, 8.3ms
image 1812/2324 /kaggle/working/split_data/val/images/24122020_152228image604843.jpg: 480x640 1 tear_crack, 4 scratchs, 1 puncture, 6.7